# Reconnaissance vocale

### 1) Fonctionnement de base

- Transformer les sons physiques en signal électrique (microphone)
- Transformer ce signal en données (convertisseur analogique-digital)
- Utilisations de différents modèles pour retranscrire l'audio en texte.

###### Hidden Markov Model (HMM)

- Division du signal sonore en fragments de 10 milisecondes
- Retranscription en vecteur de nombres (coefficients cepstraux = transformation en cosinus appliquée au spectre de puissance d'un signal.)

=>  A la sortie, on a une séquence de ces vecteurs

- Regroupement des vecteurs pour former des phonèmes (plus petite unité distinctive de segmentation dans la chaîne parlée)
- Application d'un algorithme pour détermine le(s) mot(s) le plus probable que produit une séquence de phonème.

En reconnaissance vocale moderne, on utilise des réseaux neuronaux sont utilisés pour simplifier le signal vocal avant d'appliquer le HMM.

On utilise également des détecteurs d'activités vocales, pour réduire un signal audio aux seules parties susceptibles de contenir de la parole. Cela empêche le dispositif de reconnaissance de perdre du temps à analyser des parties inutiles du signal.

### 2) Premiers pas avec SpeechRecognition

###### Avec un fichier audio clair

In [1]:
import speech_recognition as sr

In [2]:
path_harvard = "./assets/harvard.wav"

In [3]:
r = sr.Recognizer()

In [4]:
# Capturer les données à partir du fichier
harvard = sr.AudioFile(path_harvard)
with harvard as source:
    audio = r.record(source)

In [5]:
audio

Chaque instance de Recognizer dispose de sept méthodes pour reconnaître la parole à partir d'une source audio à l'aide de diverses API. Ici, on va utiliser celle de google, puisqu'elle a déjà un lien vers l'API par défaut. (Possibilité de faire 50 requêtes par jour)

In [6]:
# Appliquer le modèle
r.recognize_google(audio)

'the stale smell of old beer lingers it takes heat to bring out the odour a cold dip restores health exist a salt pickled taste fine with him as well past or my favourite exist for food is the hot cross bun'

###### Fonctionnement basique de record( )

In [7]:
# Juste les 4 premières secondes
with harvard as source:
    audio = r.record(source, duration=4)
r.recognize_google(audio)

'the stale smell of old beer lingers'

In [8]:
# tranches audio
with harvard as source:
    audio1 = r.record(source, duration=4)
    audio2 = r.record(source, duration=2.8)
    audio3 = r.record(source, duration=2.4)

r.recognize_google(audio1)

'the stale smell of old beer lingers'

In [9]:
r.recognize_google(audio2)

'it takes heat to bring out the odour'

In [10]:
r.recognize_google(audio3)

'a cold dip restores health'

In [11]:
# Audio à partir d'un point
with harvard as source:
    audio = r.record(source, offset=4, duration=3)

r.recognize_google(audio)

'it takes heat to bring out the odour'

In [12]:
# Audio à partir d'un point au milieu d'une phrase
with harvard as source:
    audio = r.record(source, offset=4.7, duration=3.3)

r.recognize_google(audio)

'exceed to bring out the older I called'

###### Audio avec du bruit

In [13]:
path_jackhammer = "./assets/jackhammer.wav"

In [14]:
jackhammer = sr.AudioFile(path_jackhammer)
with jackhammer as source:
    audio = r.record(source)

In [15]:
r.recognize_google(audio)

'smell your fingers'

In [16]:
# Ajuster au bruit ambiant
with jackhammer as source:
    r.adjust_for_ambient_noise(source)
    audio = r.record(source)

In [17]:
r.recognize_google(audio)

'still smell your fingers'

In [18]:
# Ajuster la durée du calibrage du bruit (ne sert à rien ici, trop de bruit)
with jackhammer as source:
    r.adjust_for_ambient_noise(source, duration=0.5)
    audio = r.record(source)

In [19]:
r.recognize_google(audio)

'smell your fingers'

In [20]:
# Montrer les alternatives
r.recognize_google(audio, show_all=True)

{'alternative': [{'transcript': 'smell your fingers',
   'confidence': 0.83694559},
  {'transcript': 'the smell lingers'},
  {'transcript': 'smell my fingers'},
  {'transcript': 'smell lingers'},
  {'transcript': 'the smell of your windows'}],
 'final': True}

Dans ce cas-ci, il faudrait travailler sur le preprocessing du son à part.

Petite note intéressante : j'ai pris le fichier d'un tuto utilisant la même méthode de SpeechRecognition, mais visiblement, le modèle fonctionnait mieux au moment où l'article a été écrit, et à donc été changé depuis. Puisqu'il existe une autre API de SpeechRecognitin de Google, plus professionnelle et payante... Je vous laisse à vos conclusions.

### 3) En français dans le texte

In [21]:
r = sr.Recognizer()

with sr.AudioFile('./assets/VitamineC.flac') as source:
    audio = r.record(source)

r.recognize_google(audio, language='fr-FR')

'la vitamine C mais ne dira rien'

In [22]:
r = sr.Recognizer()

with sr.AudioFile('./assets/ici-on-recolte-la-moitie-du-sel-francais_UhsGI8Y.mp3') as source:
    audio = r.record(source)

r.recognize_google(audio, language='fr-FR')

ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

Currently, SpeechRecognition supports the following file formats:

- WAV: must be in PCM/LPCM format
- AIFF
- AIFF-C
- FLAC: must be native FLAC format; OGG-FLAC is not supported